# Queries with and without Azure OpenAI

So far, you have your Search Engine loaded **from two different data sources in two diferent indexes**, on this notebook we are going to try some example queries and then use Azure OpenAI service to see if we can get a good answer for the user query.

The idea is that a user can ask a question about Computer Science (first datasource/index) or about Covid (second datasource/index), and the engine will respond accordingly.
This **Multi-Index** demo, mimics the scenario where a company loads multiple type of documents of different types and about completly different topics and the search engine must respond with the most relevant results.

## Set up variables

In [1]:
import os
import urllib
import requests
import random
import json
from collections import OrderedDict
from IPython.display import display, HTML, Markdown
from typing import List
from operator import itemgetter

# LangChain Imports needed
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from langchain_core.runnables import ConfigurableField


# Our own libraries needed
from common.prompts import DOCSEARCH_PROMPT
from common.utils import get_search_results

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

## Multi-Index Search queries

In [3]:
# Text-based Indexes that we are going to query (from Notebook 01 and 02)
index1_name = "cogsrch-index-files"
index2_name = "cogsrch-index-csv"
indexes = [index2_name, index1_name]

Try questions that you think might be answered or addressed in computer science papers in 2020-2021 or that can be addressed by medical publications about COVID in 2020-2021. Try comparing the results with the open version of ChatGPT.<br>
The idea is that the answers using Azure OpenAI only looks at the information contained on these publications.

**Example Questions you can ask**:
- What is CLP?
- How Markov chains work?
- What are some examples of reinforcement learning?
- What are the main risk factors for Covid-19?
- What medicine reduces inflamation in the lungs?
- Why Covid doesn't affect kids that much compared to adults?
- Does chloroquine really works against covid?
- Who won the 1994 soccer world cup? # This question should yield no answer if the system is correctly grounded

In [4]:
QUESTION = "What medicine reduces inflamation in the lungs?"

### Search on both indexes individually and aggragate results

#### **Note**: 
In order to standarize the indexes, **there must be 6 mandatory fields present on each index**: `id, title, name, location, chunk, chunkVector`. This is so that each document can be treated the same along the code. Also, **all indexes must have a semantic configuration**.

We are going to use Hybrid Queries: Text + Vector Search combined for optimal results!

In [5]:
agg_search_results = dict()
k = 10

for index in indexes:
    search_payload = {
        "search": QUESTION, # Text query
        "select": "id, title, name, location, chunk",
        "queryType": "semantic",
        "vectorQueries": [{"text": QUESTION, "fields": "chunkVector", "kind": "text", "k": k}], # Vector query
        "semanticConfiguration": "my-semantic-config",
        "captions": "extractive",
        "answers": "extractive",
        "count":"true",
        "top": k
    }

    r = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index + "/docs/search",
                     data=json.dumps(search_payload), headers=headers, params=params)
    print(r.status_code)

    search_results = r.json()
    agg_search_results[index]=search_results
    print("Index:", index, "Results Found: {}, Results Returned: {}".format(search_results['@odata.count'], len(search_results['value'])))

200
Index: cogsrch-index-csv Results Found: 1242, Results Returned: 10
200
Index: cogsrch-index-files Results Found: 68966, Results Returned: 10


### Display the top results (from both searches) based on the score

In [6]:
display(HTML('<h4>Top Answers</h4>'))

for index,search_results in agg_search_results.items():
    for result in search_results['@search.answers']:
        if result['score'] > 0.5: # Show answers that are at least 50% of the max possible score=1
            display(HTML('<h5>' + 'Answer - score: ' + str(round(result['score'],2)) + '</h5>'))
            display(HTML(result['text']))
            
print("\n\n")
display(HTML('<h4>Top Results</h4>'))

content = dict()
ordered_content = OrderedDict()


for index,search_results in agg_search_results.items():
    for result in search_results['value']:
        if result['@search.rerankerScore'] > 1:# Show answers that are at least 25% of the max possible score=4
            content[result['id']]={
                                    "title": result['title'],
                                    "chunk": result['chunk'], 
                                    "name": result['name'], 
                                    "location": result['location'] ,
                                    "caption": result['@search.captions'][0]['text'],
                                    "score": result['@search.rerankerScore'],
                                    "index": index
                                    }
    
#After results have been filtered we will Sort and add them as an Ordered list\n",
for id in sorted(content, key= lambda x: content[x]["score"], reverse=True):
    ordered_content[id] = content[id]
    url = str(ordered_content[id]['location']) + os.environ['BLOB_SAS_TOKEN']
    title = str(ordered_content[id]['title']) if (ordered_content[id]['title']) else ordered_content[id]['name']
    score = str(round(ordered_content[id]['score'],2))
    display(HTML('<h5><a href="'+ url + '">' + title + '</a> - score: '+ score + '</h5>'))
    display(HTML(ordered_content[id]['caption']))

### Comments on Query results

As seen above the semantic re-ranking feature of Azure AI Search service is good. It gives answers (sometimes) and also the top results with the corresponding file and the paragraph where the answers is possible located.

Let's see if we can make this better with Azure OpenAI

# Using Azure OpenAI

To use OpenAI to get a better answer to our question, the thought process is simple: let's **give the answer and the content of the documents from the search result to the GPT model as context and let it provide a better response**. This is what RAG (Retreival Augmented Generation) is about.

Now, before we do this, we need to understand a few things first:

1) Chainning and Prompt Engineering
2) Embeddings

We will use a library call **LangChain** that wraps a lot of boiler plate code.
Langchain is one library that does a lot of the prompt engineering for us under the hood, for more information see [here](https://python.langchain.com/en/latest/index.html)

In [7]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

**Important Note**: Starting now, we will utilize OpenAI models. Please ensure that you have deployed the following models within the Azure OpenAI portal:

- text-embedding-ada-002 (or newer)
- gpt-35-turbo (1106 or newer)
- gpt-4-turbo (1106 or newer)

Reference for Azure OpenAI models (regions, limits, dimensions, etc): [HERE](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models)

## A gentle intro to chaining LLMs and prompt engineering

Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step.

Azure OpenAI is a type of LLM (provider) that you can use but there are others like Cohere, Huggingface, etc.

Chains can be simple (i.e. Generic) or specialized (i.e. Utility).

* Generic — A single LLM is the simplest chain. It takes an input prompt and the name of the LLM and then uses the LLM for text generation (i.e. output for the prompt).

Here’s an example:

In [8]:
COMPLETION_TOKENS = 2500
llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], 
                      temperature=0, 
                      max_tokens=COMPLETION_TOKENS)

In [9]:
output_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant that give thorough responses to users."),
    ("user", "{input}. Give your response in {language}")
])

The | symbol is similar to a unix pipe operator, which chains together the different components feeds the output from one component as input into the next component.

In [10]:
chain = prompt | llm | output_parser

In [11]:
%%time
display(Markdown(chain.invoke({"input": QUESTION, "language": "Spanish"})))

El medicamento que reduce la inflamación en los pulmones se llama corticoesteroide inhalado. Este tipo de medicamento se utiliza comúnmente para tratar enfermedades respiratorias crónicas como el asma y la enfermedad pulmonar obstructiva crónica (EPOC). Los corticoesteroides inhalados ayudan a reducir la inflamación en las vías respiratorias y pueden aliviar los síntomas como la dificultad para respirar y la opresión en el pecho. Es importante seguir las indicaciones de su médico y utilizar el medicamento según lo prescrito para obtener los mejores resultados.

CPU times: user 29.7 ms, sys: 12.2 ms, total: 41.9 ms
Wall time: 3.45 s


**Note**: this is the first time you use OpenAI in this Accelerator, so if you get a Resource not found error, is most likely because the name of your OpenAI model deployment is different than the environmental variable set above `os.environ["GPT35_DEPLOYMENT_NAME"]`

Great!!, now you know how to create a simple prompt and use a chain in order to answer a general question using ChatGPT knowledge!. 

It is important to note that we rarely use generic chains as standalone chains. More often they are used as building blocks for Utility chains (as we will see next). Also important to notice is that we are NOT using our documents or the result of the Azure Search yet, just the knowledge of ChatGPT on the data it was trained on.

**The second type of Chains are Utility:**

* Utility — These are specialized chains, comprised of many building blocks to help solve a specific task. For example, LangChain supports some end-to-end chains (such as `create_retrieval_chain` for QnA Doc retrieval, Summarization, etc).

We will build our own specific chain in this workshop for digging deeper and solve our use case of enhancing the results of Azure AI Search.


But before dealing with the utility chain needed, let's first review the concept of Embeddings and Vector Search and RAG. 

## Embeddings and Vector Search

From the Azure OpenAI documentation ([HERE](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/embeddings?tabs=python)), An embedding is a special format of data representation that can be easily utilized by machine learning models and algorithms. The embedding is an information dense representation of the semantic meaning of a piece of text. Each embedding is a vector of floating point numbers, such that the distance between two embeddings in the vector space is correlated with semantic similarity between two inputs in the original format. For example, if two texts are similar, then their vector representations should also be similar. 

### Why Do We Need Vectors?

Vectors are essential for several reasons:

- **Semantic Richness**: They convert the semantic meaning of text into mathematical vectors, capturing nuances that simple keyword searches miss. This makes them incredibly powerful for understanding and processing language.
- **Human-like Searching**: Searching using vector distances mimics the human approach to finding information based on context and meaning, rather than relying solely on exact word matches.
- **Efficiency in Scale**: Vector representations allow for efficient handling and searching of large datasets. By reducing complex text to numerical vectors, algorithms can quickly sift through vast amounts of information.

### Understanding LLM Tokens' Context Limitation

Large Language Models (LLMs) like GPT come with a token limit for each input, which poses a challenge when dealing with lengthy documents or extensive data sets. This limitation restricts the model's ability to understand and generate responses based on the full context of the information provided. It becomes crucial, therefore, to devise strategies that can effectively manage and circumvent this limitation to leverage the full power of LLMs.

To address this challenge, the solution incorporates several key steps:

1. **Segmenting Documents**: Breaking down large documents into smaller, manageable segments.
2. **Vectorization of Chunks**: Converting these segments into vectors, making them compatible with vector-based search techniques.
3. **Hybrid Search**: Employing both vector and text search methods to pinpoint the most relevant segments in relation to the query.
4. **Optimal Context Provision**: Presenting the LLM with the most pertinent segments, ensuring a balance between detail and brevity to stay within token limits.


Our ultimate goal is to rely solely on vector indexes and hybrid searchs (vector + text). While it is possible to manually code parsers with OCR for various file types and develop a scheduler to synchronize data with the index, there is a more efficient alternative: **Azure AI Search has automated chunking strategies and vectorization**.

It's important to note that **document segmentation and vectorization have already been completed in AI Azure Search**, as seen in the `ordered_content` dictionary. This pre-processing step simplifies subsequent operations, ensuring rapid response times and adherence to the token limits of the chosen OpenAI model.


So really, our only job now is to make sure that the results from the Azure AI Search queries fit on the LLM context size, and then let it do its magic.

In [12]:
index_name = "cogsrch-index-files"
index2_name = "cogsrch-index-csv"
indexes = [index1_name, index2_name]

In order to not duplicate code, we have put many of the code used above into functions. These functions are in the `common/utils.py` and `common/prompts.py` files. This way we can use these functios in the app that we will build later.

`get_search_results()` do the multi-index search and returns the combined ordered list of documents/chunks.

In [13]:
k = 20  # play with this parameter and see the quality of the final answer
ordered_results = get_search_results(QUESTION, indexes, k=k, reranker_threshold=1)
print("Number of results:",len(ordered_results))

Number of results: 20


In [14]:
# Uncomment the below line if you want to inspect the ordered results
# ordered_results

Now let's create a Prompt Template that will ground the response only in the chunks retrieve by our hybrid AI Search.

In [15]:
template = """Answer the question thoroughly, based **ONLY** on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [16]:
%%time 
# Creation of our custom chain
chain = prompt | llm | output_parser

try:
    display(Markdown(chain.invoke({"question": QUESTION, "context": ordered_results})))
except Exception as e:
    print(e)

Based on the given context, there is no specific medicine mentioned that reduces inflammation in the lungs. The context provides information about various topics related to lung inflammation, such as nitric oxide (NO) and its contribution to lung injury, heme oxygenase-1 and carbon monoxide in pulmonary medicine, lung inflammation in influenza pneumonia, surfactant protein-D and pulmonary host defense, and the use of complementary and alternative medicines to target the host response during severe influenza. However, none of these specifically mention a medicine that reduces inflammation in the lungs.

CPU times: user 14.6 ms, sys: 2.97 ms, total: 17.6 ms
Wall time: 2.78 s


### From GPT-3.5 to GPT-4

Now let's see how the response changes if we change to GPT-4


In [17]:
llm_2 = AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=COMPLETION_TOKENS)
chain = prompt | llm_2 | output_parser

In [18]:
%%time
try:
    display(Markdown(chain.invoke({"question": QUESTION, "context": ordered_results})))
except Exception as e:
    print(e)

Based on the provided context, several substances and treatments are mentioned as having the potential to reduce inflammation in the lungs:

1. **Nitric Oxide (NO•)**: Although nitric oxide is known to have anti-microbial, anti-inflammatory, and anti-oxidant properties, its role in lung inflammation is complex. In some disease models, it contributes to lung injury, but the context does not provide conclusive evidence of its use as a medicine to reduce lung inflammation.

2. **Heme Oxygenase-1 (HO-1)** and **Carbon Monoxide (CO)**: HO-1 is an inducible stress protein that confers cytoprotection against oxidative stress and has anti-inflammatory effects. CO, a product of HO-1 activity, at low concentrations can have anti-inflammatory and anti-apoptotic effects. This suggests a potential role for HO-1/CO in reducing lung inflammation.

3. **Biochanin A**: Biochanin A is a ligand of the peroxisome proliferator activated receptors (PPAR) alpha and gamma, and the active isoflavone in Trifolium pratense (red clover). It has anti-inflammatory properties and could be used as an influenza treatment, which may imply a role in reducing lung inflammation.

4. **Fluticasone Propionate and Salmeterol**: The combination of fluticasone, a corticosteroid, and salmeterol, a long-acting beta2-agonist, is mentioned as being more effective in reducing airway hyperreactivity and inflammation than either of them alone in allergen-sensitized, RSV-infected mice.

5. **Corticosteroids**: Corticosteroids are mentioned in the context of various treatment therapies tried in the management of patients with ARDS, which includes reducing lung inflammation.

6. **Surfactant Protein-D (SP-D)**: SP-D participates in the innate response to inhaled microorganisms and contributes to immune and inflammatory regulation within the lung. Deficient accumulation of SP-D might contribute to the pathogenesis of human lung diseases, suggesting its role in managing lung inflammation.

7. **Complementary and Alternative Medicines**: Certain Chinese herbs like Angelica sinensis (Dang Gui) and Salvia miltiorrhiza (Danshen) have been shown to reduce inflammation through inhibition of inflammatory cytokines like High Mobility Group Box 1 protein (HMGB1).

These substances and treatments highlight the diverse approaches being investigated or used to reduce lung inflammation. However, it's important to note that the effectiveness, safety, and mechanisms of action of these treatments can vary, and more research may be needed to fully understand their potential as medicines for reducing lung inflammation.

CPU times: user 17 ms, sys: 3.45 ms, total: 20.4 ms
Wall time: 1min 33s


#### As we can see, the model selection MATTERS!

We will dive deeper into this later, but for now, **look at the diference between GPT3.5 and GPT4, in quality and in response time**.

# Improving the Prompt and adding citations

We could see above that the answer given by GPT3.5 was very simple compared to GPT4, even when the prompt says "thorough responses to users". We also could see that there is no citations or references. **How do we know if the answer is grounded on the context or not?**

Let's see if these two issues can be improved by Prompt Engineering.<br>
On `common/prompts.py` we created a prompt called `DOCSEARCH_PROMPT` check it out!

Let's also create a custom Retriever class so we can plug it in easily within the chain building. 
Note: we can also use the Azure AI Search retriever class [HERE](https://python.langchain.com/docs/integrations/vectorstores/azuresearch), however we want to create a custom Retriever for the following reasons:

1) We want to do multi-index searches in one call
2) Easier to teach complex concepts of LangChain in this notebook
3) We want to use the REST API vs the Python Azure Search SDK

In [22]:
class CustomRetriever(BaseRetriever):
    
    topK : int
    reranker_threshold : int
    indexes: List
    sas_token: str = None
    
    def _get_relevant_documents(self, query: str) -> List[Document]:
        
        ordered_results = get_search_results(query, self.indexes, k=self.topK, 
                                             reranker_threshold=self.reranker_threshold, 
                                             sas_token=self.sas_token)
        top_docs = []
        for key,value in ordered_results.items():
            location = value["location"] if value["location"] is not None else ""
            top_docs.append(Document(page_content=value["chunk"], metadata={"source": location, "score":value["score"]}))

        return top_docs

In [23]:
# Create the retriever
retriever = CustomRetriever(indexes=indexes, topK=k, reranker_threshold=1, sas_token=os.environ['BLOB_SAS_TOKEN'])

In [24]:
# Test retreiver
results = retriever.get_relevant_documents(QUESTION)
len(results)

20

In [25]:
# We can create now a dynamically configurable llm object that can change the model at runtime
dynamic_llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], 
                              temperature=0.5, max_tokens=COMPLETION_TOKENS).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="model"),
    # This sets a default_key.
    # If we specify this key, the default LLM  (initialized above) will be used
    default_key="gpt35",
    # This adds a new option, with name `gpt4`
    gpt4=AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], 
                         temperature=0.5, max_tokens=COMPLETION_TOKENS),
    # You can add more configuration options here
)

In [26]:
# Declaration of the chain with the dynamic llm and the new prompt
configurable_chain = (
    {
        "context": itemgetter("question") | retriever, # Passes the question to the retriever and the results are assign to context
        "question": itemgetter("question")
    }
    | DOCSEARCH_PROMPT  # Passes the input variables above to the prompt template
    | dynamic_llm   # Passes the finished prompt to the LLM
    | StrOutputParser()  # converts the output (Runnable object) to the desired output (string)
)

In [27]:
%%time

try:
    display(Markdown(configurable_chain.with_config(configurable={"model": "gpt35"}).invoke({"question": QUESTION})))
except Exception as e:
    print(e)

Heme oxygenase-1 (HO-1) is a medicine that has been shown to reduce inflammation in the lungs<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC193681/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D" target="_blank">[2]</a></sup>. HO-1 is an inducible stress protein that has anti-inflammatory effects and limits tissue damage in response to proinflammatory stimuli. It is involved in the degradation of heme and catalyzes the conversion of heme to its metabolites, such as bilirubin IXα, ferrous iron, and carbon monoxide (CO). The enzymatic reaction products of HO-1, including CO, have been found to exhibit anti-inflammatory and anti-apoptotic effects<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC193681/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D" target="_blank">[2]</a></sup>. The administration of CO at low concentrations can substitute for HO-1 in exerting these anti-inflammatory and anti-apoptotic effects. Therefore, HO-1 and its reaction products, such as CO, have been studied for their potential therapeutic implications in pulmonary diseases and inflammation in the lungs<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC193681/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D" target="_blank">[2]</a></sup>.

CPU times: user 66.1 ms, sys: 11.7 ms, total: 77.8 ms
Wall time: 12.3 s


As seen above, we were able to improve the quality and breath of the answer and add citations with only prompt engineering!

Let's try again GPT-4

In [28]:
%%time
try:
    display(Markdown(configurable_chain.with_config(configurable={"model": "gpt4"}).invoke({"question": QUESTION})))
except Exception as e:
    print(e)

Several documents provide information on various substances and treatments that can reduce inflammation in the lungs:

1. **Heme Oxygenase-1 (HO-1)**: HO-1 is an inducible stress protein that confers cytoprotection against oxidative stress both in vitro and in vivo. It has anti-inflammatory effects, limiting tissue damage in response to proinflammatory stimuli and preventing allograft rejection after transplantation. The anti-inflammatory and anti-apoptotic effects of HO-1 can be substituted by administering carbon monoxide (CO) at low concentrations, suggesting a role for CO as a key mediator of HO-1 function<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC193681/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D" target="_blank">[2]</a></sup>.

2. **Aerosolized Bacterial Lysate**: In a study, mice exposed to an aerosolized bacterial lysate prior to challenge with live influenza virus exhibited augmented inflammation but also showed resistance to host mortality and reduced viral titers. This suggests that innate immune resistance to influenza can be effectively stimulated, and that ineffective rather than excessive inflammation is a major cause of mortality in influenza pneumonia<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2613561/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D" target="_blank">[3]</a></sup>.

3. **Clarithromycin (CAM)**: CAM is an antimicrobial agent and an immunomodulator that significantly suppressed RSV-induced production of interleukin-6, interleukin-8, and RANTES. CAM also suppressed RSV-induced PAF receptor expression and adhesion of S. pneumoniae cells to host cells, thus potentially suppressing immunological disorders and preventing secondary bacterial infections during RSV infection<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3384978/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D" target="_blank">[4]</a></sup>.

4. **Surfactant Protein-D (SP-D)**: SP-D is involved in the innate immune response to inhaled microorganisms and contributes to immune and inflammatory regulation within the lung. SP-D-deficient transgenic mice show abnormal accumulations of surfactant lipids, and respond abnormally to challenge with respiratory viruses and bacterial lipopolysaccharides, suggesting that SP-D plays a role in modulating lung inflammation<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC59549/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D" target="_blank">[5]</a></sup>.

5. **Chinese Herbs and PPAR Ligands**: Chinese herbs like Angelica sinensis and Salvia miltiorrhiza, as well as the PPAR ligands like biochanin A and gemfibrozil, have been shown to have anti-inflammatory properties and could be considered as potential candidates for influenza treatments<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2892358/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D" target="_blank">[6]</a></sup>.

6. **Fluticasone and Salmeterol**: In a study, fluticasone propionate (FP) and salmeterol (Sal) alone, and in combination (FPS), were tested on RSV-induced airway inflammation. The combination treatment (FPS) was more effective in decreasing airway hyperreactivity and inflammation than either FP or Sal alone in allergen-sensitized, RSV-infected mice<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1488829/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D" target="_blank">[8]</a></sup>.

7. **Human Embryonic Stem Cells**: Human embryonic stem cells differentiated under mesoderm-inducing conditions have therapeutic properties in sepsis-induced lung injury by reducing lung inflammation and edema, as well as the production of proinflammatory cytokines<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3069906/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D" target="_blank">[9]</a></sup>.

These are just some examples of substances and treatments that have been studied for their potential to reduce lung inflammation. It is important to note that the effectiveness and applicability of these treatments can vary based on the specific disease context and should be guided by clinical evidence and medical advice.

CPU times: user 56.7 ms, sys: 7 ms, total: 63.8 ms
Wall time: 3min 6s


#### As you can see the answer from GPT4 is richer, and includes all the relevant chunks. GPT3.5 tends to focus in the first and last chunks only

## Adding Streaming to improve user experience and performance

It is obvious by now that **GPT4 answers are better quality than GPT3.5**. None are incorrect, but GPT4 is better at understanding the context, following the prompt instructions and on giving a comprehensive answer.

One way to make GPT4 look faster is to stream the answer, so the user can see the response as it is typed. To do this, we just simply need to call the method `stream` instead of `invoke`. More on Streaming and Callbacks in later notebooks, but for now, this is one simple way to do it:

In [29]:
for chunk in configurable_chain.with_config(configurable={"model": "gpt4"}).stream({"question": QUESTION}):
    print(chunk, end="", flush=True)

Several medicines and treatments have been identified that may reduce inflammation in the lungs:

1. **Heme Oxygenase-1 (HO-1)**: This inducible stress protein confers cytoprotection against oxidative stress and has anti-inflammatory effects that can limit tissue damage in response to proinflammatory stimuli. HO-1 can prevent allograft rejection after transplantation and reduce inflammation. Administration of carbon monoxide (CO) at low concentrations can substitute for HO-1 with respect to anti-inflammatory effects, suggesting a role for CO as a mediator of HO-1 function in the lungs<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC193681/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D" target="_blank">[2]</a></sup>.

2. **Clarithromycin**: This antimicrobial agent, also known as an immunomodulator, significantly suppressed RSV-induced production of proinflammatory cytokines and P

# Summary
##### By using OpenAI, the answers to user questions are way better than taking just the results from Azure AI Search. So the summary is:
- Utilizing Azure AI Search, we conduct a multi-index hybrid search that identifies the top chunks of documents from each index.
- Subsequently, Azure OpenAI utilizes these extracted chunks as context, comprehends the content, and employs it to deliver optimal answers.
- Best of two worlds!

##### Important observations on this notebook:

1) Answers with GPT-3.5 are less quality but way faster
2) Answers with GPT-3.5 sometimes failed on provinding citations in the right format
3) Answers with GPT-4 are great quality but way slower
4) Answers with GPT-4 always provide good and diverse citations in the right format
5) Streaming the answers improves the user experience big time!

# NEXT
In the next notebook, we are going to see how we can treat complex and large documents separately, also using Vector Search